In [1]:
import tensorflow as tf
from object_detection.utils import dataset_util
import json
from PIL import Image
import os
import io
import sys

In [2]:
#Sign Detection

In [3]:
img_path = "sign_detection/images/test/"
text_img_path = "sign_detection/images/test/"
annot_path = "sign_detection/annotations/test_annotations.json"
tfrecord_path = "sign_detection/annotations/test.record"

In [3]:
flags = tf.app.flags
flags.DEFINE_string('output_path', '', 'sign_detection/annotations/test.record')
FLAGS = flags.FLAGS

In [4]:
def create_tf_example(annot):
    # TODO(user): Populate the following variables from your example.
    
    fname = annot['filename']
    
    with tf.gfile.GFile(os.path.join(img_path, '{}'.format(fname)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
#     print(type(encoded_jpg))
#     print(type(encoded_jpg_io))
#     print(type(image))

    #height = None # Image height
    #width = None # Image width
    filename = fname.encode('utf8') # Filename of the image. Empty if image is not from file
    encoded_image_data = encoded_jpg # Encoded image bytes
    image_format = b'jpg' # b'jpeg' or b'png'
    
    xmins, ymins, xmaxs, ymaxs = [], [], [], []
    classes_text = []
    classes = []
    count = 0
    for box in annot['regions']:
        if box["shape_attributes"]["name"] == "rect":
            count += 1
            xmins.append(box["shape_attributes"]["x"])
            ymins.append(box["shape_attributes"]["y"])
            xmaxs.append(box["shape_attributes"]["x"] + box["shape_attributes"]["width"])
            ymaxs.append(box["shape_attributes"]["y"] + box["shape_attributes"]["height"])
            classes_text.append('sign'.encode('utf8'))
            classes.append(1)
            
    if len(xmins) == 0:
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []
    else:
        xmins = [xmin / width for xmin in xmins] # List of normalized left x coordinates in bounding box (1 per box)
        xmaxs = [xmax / width for xmax in xmaxs] # List of normalized right x coordinates in bounding box
                 # (1 per box)
        ymins = [ymin / height for ymin in ymins] # List of normalized top y coordinates in bounding box (1 per box)
        ymaxs = [ymax / height for ymax in ymaxs] # List of normalized bottom y coordinates in bounding box
                 # (1 per box)
        #classes_text = ['sign'.encode('utf8')] # List of string class name of bounding box (1 per box)
        #classes = [1] # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    
    #if count > 2:
    #    print(tf_example)
    return tf_example


In [5]:
writer = tf.python_io.TFRecordWriter(tfrecord_path)

# TODO(user): Write code to read in your dataset to examples variable

with open(annot_path) as rb:
    annots = json.load(rb)

for annot in annots:
    tf_example = create_tf_example(annot)
    break
    writer.write(tf_example.SerializeToString())

writer.close()

In [6]:
print(len(annots))

301


In [7]:
#Text Detection

In [8]:
img_path = "sign_detection/images/test/"
text_img_path = "sign_detection/images_text_detection/test/"
annot_path = "sign_detection/annotations/test_annotations.json"
tfrecord_path = "sign_detection/annotations/test_text.record"

In [9]:
flags = tf.app.flags
flags.DEFINE_string('output_path', '', 'sign_detection/annotations/test_text.record')
FLAGS = flags.FLAGS

In [10]:
def create_tf_example(annot):
    fname = annot['filename']
    
    with tf.gfile.GFile(os.path.join(img_path, '{}'.format(fname)), 'rb') as fid:
        encoded_img = fid.read()
    encoded_img_io = io.BytesIO(encoded_img)
    
    sign_count = 0
    tf_list = []
    
    for box_sign in annot['regions']:
        if box_sign["shape_attributes"]["name"] == "rect":
            xmin_sign = box_sign["shape_attributes"]["x"]
            ymin_sign = box_sign["shape_attributes"]["y"]
            xmax_sign = box_sign["shape_attributes"]["x"] + box_sign["shape_attributes"]["width"]
            ymax_sign = box_sign["shape_attributes"]["y"] + box_sign["shape_attributes"]["height"]
            
            fname_box = fname[:-4] + "_" + str(sign_count) + ".jpg"
            
            image_sign = Image.open(encoded_img_io)
            cropped_img = image_sign.crop((xmin_sign, ymin_sign, xmax_sign, ymax_sign))
            cropped_img.save(text_img_path + fname_box)
            
            with tf.gfile.GFile(os.path.join(text_img_path, '{}'.format(fname_box)), 'rb') as fid:
                encoded_jpg = fid.read()
            encoded_jpg_io = io.BytesIO(encoded_jpg)
            image = Image.open(encoded_jpg_io)
            width, height = image.size
            
            filename = fname_box.encode('utf8') # Filename of the image. Empty if image is not from file
            encoded_image_data = encoded_jpg # Encoded image bytes
            image_format = b'jpg' # b'jpeg' or b'png'
            
            xmins, ymins, xmaxs, ymaxs = [], [], [], []
            classes_text = []
            classes = []
            
            #print("Sign Count:", sign_count)
            
            for box in annot['regions']:
                if box["shape_attributes"]["name"] == "polygon":
                    x_points = box["shape_attributes"]["all_points_x"]
                    y_points = box["shape_attributes"]["all_points_y"]
                    
                    x_points = [int(xp.split('-')[0]) if type(xp) == str else xp for xp in x_points]
                    y_points = [int(yp.split('-')[0]) if type(yp) == str else yp for yp in y_points]
                    
                    xmin = min(x_points)
                    xmax = max(x_points)
                    ymin = min(y_points)
                    ymax = max(y_points)
                    
                    #center_x = (xmin + xmax)/2
                    #center_y = (ymin + ymax)/2
                    
                    #print(x_points, y_points)
                    #print("Poly:",xmin, xmax, ymin, ymax)
                    #print("Sign:",xmin_sign, xmax_sign, ymin_sign, ymax_sign)
                    
                    if (xmin >= xmin_sign and ymin >= ymin_sign and xmax <= xmax_sign and ymax <= ymax_sign):
                        xmin_rel = abs(xmin - xmin_sign)
                        ymin_rel = abs(ymin - ymin_sign)
                        xmax_rel = abs(xmax - xmin_sign)
                        ymax_rel = abs(ymax - ymin_sign)
                        
                        xmins.append(xmin_rel)
                        ymins.append(ymin_rel)
                        xmaxs.append(xmax_rel)
                        ymaxs.append(ymax_rel)
                        classes_text.append('text'.encode('utf8'))
                        classes.append(1)
                            
            if len(xmins) == 0:
                xmins = []
                xmaxs = []
                ymins = []
                ymaxs = []
                classes_text = []
                classes = []
            else:
                xmins = [xmin / width for xmin in xmins] # List of normalized left x coordinates in bounding box (1 per box)
                xmaxs = [xmax / width for xmax in xmaxs] # List of normalized right x coordinates in bounding box
                         #(1 per box)
                ymins = [ymin / height for ymin in ymins] # List of normalized top y coordinates in bounding box (1 per box)
                ymaxs = [ymax / height for ymax in ymaxs] # List of normalized bottom y coordinates in bounding box
                         #(1 per box)
                    
                tf_example = tf.train.Example(features=tf.train.Features(feature={
                  'image/height': dataset_util.int64_feature(height),
                  'image/width': dataset_util.int64_feature(width),
                  'image/filename': dataset_util.bytes_feature(filename),
                  'image/source_id': dataset_util.bytes_feature(filename),
                  'image/encoded': dataset_util.bytes_feature(encoded_image_data),
                  'image/format': dataset_util.bytes_feature(image_format),
                  'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                  'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                  'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                  'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                  'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
                  'image/object/class/label': dataset_util.int64_list_feature(classes),
                }))
                
                tf_list.append(tf_example)
            
            #print(tf_example)
            
            #tf_list.append(tf_example)
            
            sign_count += 1
            
    print("Image:", fname, "Signs:", sign_count)
    
#     if sign_count > 1:
#         print(tf_list)
   
    return tf_list

In [11]:
writer = tf.python_io.TFRecordWriter(tfrecord_path)

# TODO(user): Write code to read in your dataset to examples variable

with open(annot_path) as rb:
    annots = json.load(rb)

count = 0
for annot in annots:
    tf_list = create_tf_example(annot)
    #break
    for tf_example in tf_list:
        writer.write(tf_example.SerializeToString())
        count += 1

writer.close()

print("Count:", count)

Image: 1382.jpg Signs: 3
Image: 286.jpg Signs: 2
Image: 941.jpg Signs: 2
Image: 519.jpg Signs: 2
Image: 196.jpg Signs: 1
Image: 1385.jpg Signs: 3
Image: 352.jpg Signs: 1
Image: 530.jpg Signs: 1
Image: 1396.jpg Signs: 2
Image: 747.jpg Signs: 2
Image: 1042.jpg Signs: 2
Image: 1328.jpg Signs: 1
Image: 191.jpg Signs: 1
Image: 1209.jpg Signs: 1
Image: 738.jpg Signs: 1
Image: 293.jpg Signs: 2
Image: 807.jpg Signs: 2
Image: 940.jpg Signs: 2
Image: 1079.jpg Signs: 1
Image: 635.jpg Signs: 1
Image: 522.jpg Signs: 1
Image: 1367.jpg Signs: 3
Image: 515.jpg Signs: 1
Image: 719.jpg Signs: 1
Image: 306.jpg Signs: 3
Image: 1295.jpg Signs: 3
Image: 898.jpg Signs: 1
Image: 287.jpg Signs: 2
Image: 935.jpg Signs: 2
Image: 1432.jpg Signs: 1
Image: 1027.jpg Signs: 2
Image: 727.jpg Signs: 1
Image: 1041.jpg Signs: 2
Image: 80.jpg Signs: 1
Image: 76.jpg Signs: 1
Image: 1283.jpg Signs: 2
Image: 1402.jpg Signs: 2
Image: 359.jpg Signs: 1
Image: 292.jpg Signs: 2
Image: 804.jpg Signs: 1
Image: 1370.jpg Signs: 3
Ima

In [12]:
print(len(annots))

301
